In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

import pickle

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
with open(r"C:\Users\tom_r\Desktop\Machine-Learning\RF_spectrum_analysis\subset.pkl", 'rb') as file:
    dataset = pickle.load(file)

In [5]:
for k, v in dataset.items():
    print(f"{k}: shape {v.shape}, dtype {v.dtype}")

X: shape (532480, 1024, 2), dtype float16
y: shape (532480,), dtype int64
snr: shape (532480,), dtype int64


In [6]:
X_train, X_text, y_train, y_test = train_test_split(dataset['X'], dataset['y'], test_size=0.2, random_state=44)

In [7]:
X_tensor = torch.tensor(X_train, dtype=torch.float32)  
X_tensor = X_tensor.permute(0, 2, 1)  # (N, 2, 1024)

In [8]:
print(len(X_train))
X_train[0].shape

425984


(1024, 2)

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer1 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.max_pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv_layer3 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.max_pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32384, 128)
        self.fc2 = nn.Linear(128, 5) # 5 classes for the subset

    def forward(self, x):
        x = self.conv_layer1(x)
        x = F.relu(x)
        x = self.conv_layer2(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv_layer3(x)
        x = F.relu(x)
        x = self.conv_layer4(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.flatten(x)
        print(x.shape)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return x


In [17]:
m = CNN()
output = m.forward(X_tensor[0].unsqueeze(0))  # now shape: (1, 2, 1024)
output.shape

torch.Size([1, 32384])


torch.Size([1, 5])

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)